In [21]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
import re

In [22]:
def strip_emoji(text):
        emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
        return emoji_pattern.sub('EMOJI', text)

In [23]:
def extractor(query: str, csvname: str):
    tweets = []

    for tweet in sntwitter.TwitterSearchScraper(query=query).get_items():
        tweets.append(
            [
                tweet.date,
                tweet.id,
                tweet.content,
                tweet.user.username,
            ]
        )
    df = pd.DataFrame(tweets, columns=["Datetime", "Tweet Id", "Text", "Username"])
    df["no_http"] = [re.sub(r"http\S+", "", tweet) for tweet in df["Text"]]
    df["no_arroba"] = [re.sub(r"[@]", "", x) for x in df["no_http"]]
    df["no_hastag"] = [re.sub(r"#", "", x) for x in df["no_arroba"]]
    df["no_espaço"] = [re.sub(r"\n|\t|\r", " ", x) for x in df["no_hastag"]]
    df['texto_processado'] = df['no_espaço'].apply(strip_emoji)

    df.to_parquet(csvname,engine='fastparquet',compression='gzip')


In [24]:
extractor(
    "Vasco lang:pt until:2022-11-06 since:2022-11-05",
    "data_vasco.parquet",
)

In [25]:
df = pd.read_parquet("data_vasco.parquet")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12756 entries, 0 to 12755
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   Datetime          12756 non-null  datetime64[ns, UTC]
 1   Tweet Id          12756 non-null  int64              
 2   Text              12756 non-null  object             
 3   Username          12756 non-null  object             
 4   no_http           12756 non-null  object             
 5   no_arroba         12756 non-null  object             
 6   no_hastag         12756 non-null  object             
 7   no_espaço         12756 non-null  object             
 8   texto_processado  12756 non-null  object             
dtypes: datetime64[ns, UTC](1), int64(1), object(7)
memory usage: 897.0+ KB


In [27]:
df.isna().sum()

Datetime            0
Tweet Id            0
Text                0
Username            0
no_http             0
no_arroba           0
no_hastag           0
no_espaço           0
texto_processado    0
dtype: int64

In [28]:
df.head(30)

,Datetime,Tweet Id,Text,Username,no_http,no_arroba,no_hastag,no_espaço,texto_processado
0,2022-11-05 23:59:48+00:00,1589044844046864386,@FCPorto A frase abaixo dita por Cyro Aranha g...,Doraroxo23,@FCPorto A frase abaixo dita por Cyro Aranha g...,FCPorto A frase abaixo dita por Cyro Aranha gr...,FCPorto A frase abaixo dita por Cyro Aranha gr...,FCPorto A frase abaixo dita por Cyro Aranha gr...,FCPorto A frase abaixo dita por Cyro Aranha gr...
1,2022-11-05 23:59:43+00:00,1589044821922312192,@i94hao @VascodaGama me ajuda pfv...,identidadezerol,@i94hao @VascodaGama me ajuda pfv...,i94hao VascodaGama me ajuda pfv...,i94hao VascodaGama me ajuda pfv...,i94hao VascodaGama me ajuda pfv...,i94hao VascodaGama me ajuda pfv...
2,2022-11-05 23:59:41+00:00,1589044815194652673,"Na moral, estou a todo instante pensando na me...",WesleyL52035066,"Na moral, estou a todo instante pensando na me...","Na moral, estou a todo instante pensando na me...","Na moral, estou a todo instante pensando na me...","Na moral, estou a todo instante pensando na me...","Na moral, estou a todo instante pensando na me..."
3,2022-11-05 23:59:33+00:00,1589044780054777856,"Pessoal, amanhã, às 15h, tem @VascodaGama e @F...",LMC_jornalista,"Pessoal, amanhã, às 15h, tem @VascodaGama e @F...","Pessoal, amanhã, às 15h, tem VascodaGama e Fla...","Pessoal, amanhã, às 15h, tem VascodaGama e Fla...","Pessoal, amanhã, às 15h, tem VascodaGama e Fla...","Pessoal, amanhã, às 15h, tem VascodaGama e Fla..."
4,2022-11-05 23:59:31+00:00,1589044772949618688,@CristiFreitasS @TNTSportsBR Irmão quando você...,yuriikelviinn11,@CristiFreitasS @TNTSportsBR Irmão quando você...,CristiFreitasS TNTSportsBR Irmão quando vocês ...,CristiFreitasS TNTSportsBR Irmão quando vocês ...,CristiFreitasS TNTSportsBR Irmão quando vocês ...,CristiFreitasS TNTSportsBR Irmão quando vocês ...
5,2022-11-05 23:59:24+00:00,1589044744931667968,@yesbruno @WandersonLucin8 @FabianoAlvesC @Rec...,LuanLessa14,@yesbruno @WandersonLucin8 @FabianoAlvesC @Rec...,yesbruno WandersonLucin8 FabianoAlvesC Recibo_...,yesbruno WandersonLucin8 FabianoAlvesC Recibo_...,yesbruno WandersonLucin8 FabianoAlvesC Recibo_...,yesbruno WandersonLucin8 FabianoAlvesC Recibo_...
6,2022-11-05 23:59:20+00:00,1589044728632573952,"@pedrosa Fala, Pedrosa! saiu a provável escala...",raaycunnha,"@pedrosa Fala, Pedrosa! saiu a provável escala...","pedrosa Fala, Pedrosa! saiu a provável escalaç...","pedrosa Fala, Pedrosa! saiu a provável escalaç...","pedrosa Fala, Pedrosa! saiu a provável escalaç...","pedrosa Fala, Pedrosa! saiu a provável escalaç..."
7,2022-11-05 23:59:20+00:00,1589044725134557184,@wesley_curty Vasco é o clube que mais incomod...,ShalomA88725111,@wesley_curty Vasco é o clube que mais incomod...,wesley_curty Vasco é o clube que mais incomoda...,wesley_curty Vasco é o clube que mais incomoda...,wesley_curty Vasco é o clube que mais incomoda...,wesley_curty Vasco é o clube que mais incomoda...
8,2022-11-05 23:59:17+00:00,1589044714677751809,@animefla ceara por não ter rebaixado o vasco,arrscaetta,@animefla ceara por não ter rebaixado o vasco,animefla ceara por não ter rebaixado o vasco,animefla ceara por não ter rebaixado o vasco,animefla ceara por não ter rebaixado o vasco,animefla ceara por não ter rebaixado o vasco
9,2022-11-05 23:59:17+00:00,1589044713679904775,@SportsCenterBR Ituano 2x1 vasco,BellagMarc,@SportsCenterBR Ituano 2x1 vasco,SportsCenterBR Ituano 2x1 vasco,SportsCenterBR Ituano 2x1 vasco,SportsCenterBR Ituano 2x1 vasco,SportsCenterBR Ituano 2x1 vasco
